Copyright (c) Microsoft Corporation.
Licensed under the MIT License.

# Abstractive Summarization using BertSumAbs on CNN/DailyMails Dataset

## Summary

This notebook demonstrates how to fine tune BERT for abstractive text summarization. Utility functions and classes in the NLP Best Practices repo are used to facilitate data preprocessing, model training, model scoring, result postprocessing, and model evaluation.

### Abstractive Summarization
Abstractive summarization is the task of taking an input text and summarizing its content in a shorter output text. In contrast to extractive summarization, abstractive summarization doesn't take sentences directly from the input text, instead, rephrases the input text.

### BertSumAbs

BertSumAbs refers to an BERT-based abstractive summarization algorithm  in [Text Summarization with Pretrained Encoders](https://arxiv.org/abs/1908.08345) with [published examples](https://github.com/nlpyang/PreSumm). It uses the pretrained BERT model as encoder and finetune both encoder and decoder on a specific labeled summarization dataset like [CNN/DM dataset](https://github.com/harvardnlp/sent-summary). 

The figure below shows the comparison of architecture of the original BERT model (left) and BERTSUM (right), which BertSumAbs is built upon. For BERTSUM, a input document is split into sentences, and [CLS] and [SEP] tokens are inserted before and after each sentence. This resulting sequence is followed by the summation of three kinds of embeddings for each token before feeding into the transformer layers. The positional embedding used in BertSumAbs enables input length of more than 512, which is the  maximum input length for BERT model. 

It should be noted that the architecture only shows the encoder part. For decoder, BertSumAbs also uses a transformer with multiple layers and random initialization. As pretrained weights are used in the encoder, there is a mismatch in encoder and decoder which may result in unstable finetuning. Therefore, in fine tuning, BertSumAbs uses seperate optimizers for encoder and decoder, each uses its own scheduling. In text generation, techniques like trigram blocking and beam search can be used to improve model accuracy.
<img src="https://nlpbp.blob.core.windows.net/images/BertForSummarization.PNG">


## Before you start

It's recommended to run this notebook on GPU machines as it's very computationally intensive. Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of steps. If QUICK_RUN = False, the notebook takes about 5 hours to run on a VM with 4 16GB NVIDIA V100 GPUs. Finetuning costs around 1.5 hours and inferecing costs around 3.5 hour.  Better performance can be achieved by increasing the MAX_STEPS.

* **ROUGE Evalation**: To run rouge evaluation, please refer to the section of compute_rouge_perl in [summarization_evaluation.ipynb](./summarization_evaluation.ipynb) for setup.

* **Distributed Training**:
Please note that the jupyter notebook only allows to use pytorch [DataParallel](https://pytorch.org/docs/master/nn.html#dataparallel). Faster speed and larger batch size can be achieved with pytorch [DistributedDataParallel](https://pytorch.org/docs/master/notes/ddp.html)(DDP). Script [abstractive_summarization_bertsum_cnndm_distributed_train.py](./abstractive_summarization_bertsum_cnndm_distributed_train.py) shows an example of how to use DDP.

* **Mixed Precision Training**:
Please note that by default this notebook doesn't use mixed precision training. Faster speed and larger batch size can be achieved when you set FP16 to True. Refer to  https://nvidia.github.io/apex and https://github.com/nvidia/apex) for details to use mixed precision training. Check the GPU model on your machine to see if it allows mixed precision training. Please also note that mixed precision inferencing is also enabled in the prediciton utility function. When you use mixed precision training and/or inferencing, the model performance can be slightly worse than the full precision mode.

In [1]:
QUICK_RUN = False

In [2]:
import os
os.getcwd()

'/home/ge75zam2/german-bertabs/examples/text_summarization'

In [3]:
import os
import shutil
import sys
from tempfile import TemporaryDirectory
import torch

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.models.transformers.abstractive_summarization_bertsum import (
    BertSumAbs,
    BertSumAbsProcessor,
    validate
)

from utils_nlp.eval import compute_rouge_python

from utils_nlp.models.transformers.datasets import SummarizationDataset
import nltk
from nltk import tokenize

import pandas as pd
import pprint
import scrapbook as sb

In [4]:
import sys
sys.path

['/home/ge75zam2/german-bertabs',
 '/home/ge75zam2/miniconda3/envs/finetune/lib/python36.zip',
 '/home/ge75zam2/miniconda3/envs/finetune/lib/python3.6',
 '/home/ge75zam2/miniconda3/envs/finetune/lib/python3.6/lib-dynload',
 '',
 '/home/ge75zam2/miniconda3/envs/finetune/lib/python3.6/site-packages',
 '/home/ge75zam2/miniconda3/envs/finetune/lib/python3.6/site-packages/IPython/extensions',
 '/home/ge75zam2/.ipython']

## Data Preprocessing

The dataset we used for this notebook is CNN/DM dataset which contains the documents and accompanying questions from the news articles of CNN and Daily mail. The highlights in each article are used as summary. The dataset consits of ~289K training examples, ~11K valiation examples and ~11K test examples. The length of the news articles is 781 tokens on average and the summaries are of 3.75 sentences and 56 tokens on average.

The significant part of data preprocessing only involve splitting the input document into sentences.

In [5]:
# the data path used to save the downloaded data file
#DATA_PATH = TemporaryDirectory().name
# The number of lines at the head of data file used for preprocessing. -1 means all the lines.
TOP_N = 100
if not QUICK_RUN:
    TOP_N = -1

In [6]:
#train_dataset, test_dataset = CNNDMSummarizationDataset(
#    top_n=TOP_N, local_cache_path=DATA_PATH, prepare_extractive=False
#)

from utils_nlp.dataset.swiss import SwissSummarizationDataset
train_dataset, validation_dataset, test_dataset = SwissSummarizationDataset( top_n=TOP_N, validation=True)


In [7]:
len(train_dataset), len(validation_dataset), len(test_dataset)

(85500, 9500, 5000)

## Model Finetuning

In [8]:
# notebook parameters
# the cache path
CACHE_PATH = '/home/ge75zam2/finetuning'

# model parameters
MODEL_NAME = "bert-base-german-cased"
MAX_POS = 768
MAX_SOURCE_SEQ_LENGTH = 640
MAX_TARGET_SEQ_LENGTH = 140

# mixed precision setting. To enable mixed precision training, follow instructions in SETUP.md.
FP16 = False
if FP16:
    FP16_OPT_LEVEL = "O2"

# fine-tuning parameters
# batch size, unit is the number of tokens
BATCH_SIZE_PER_GPU = 1


# GPU used for training
NUM_GPUS = torch.cuda.device_count()

if NUM_GPUS > 0:
    BATCH_SIZE = NUM_GPUS * BATCH_SIZE_PER_GPU
else:
    BATCH_SIZE = 1


# Learning rate
LEARNING_RATE_BERT = 5e-4 / 2.0
LEARNING_RATE_DEC = 0.05 / 2.0


# How often the statistics reports show up in training, unit is step.
REPORT_EVERY = 100
SAVE_EVERY = 1000

# total number of steps for training
MAX_STEPS = 1000

if not QUICK_RUN:
    MAX_STEPS = 10e6

WARMUP_STEPS_BERT = 2000
WARMUP_STEPS_DEC = 1000

In [9]:
from utils_nlp.common.pytorch_utils import (
    compute_training_steps,
    get_amp,
    get_device,
    move_model_to_device,
    parallelize_model,
)

get_device(NUM_GPUS)

(device(type='cuda'), 8)

In [10]:
# processor which contains the colloate function to load the preprocessed data
processor = BertSumAbsProcessor(model_name=MODEL_NAME, cache_dir=CACHE_PATH, max_src_len=MAX_SOURCE_SEQ_LENGTH, max_tgt_len=MAX_TARGET_SEQ_LENGTH)
# summarizer
summarizer = BertSumAbs(
    processor, model_name=MODEL_NAME, cache_dir=CACHE_PATH, max_pos_length=MAX_POS
)

In [11]:
print(BATCH_SIZE_PER_GPU*NUM_GPUS)
print(MAX_STEPS)

8
10000000.0


In [12]:
def validation_function(summarizer):
    return validate(summarizer, validation_dataset, language='en')

In [ ]:
summarizer.fit(
    train_dataset,
    num_gpus=NUM_GPUS,
    batch_size=BATCH_SIZE,
    max_steps=MAX_STEPS,
    learning_rate_bert=LEARNING_RATE_BERT,
    learning_rate_dec=LEARNING_RATE_DEC,
    warmup_steps_bert=WARMUP_STEPS_BERT,
    warmup_steps_dec=WARMUP_STEPS_DEC,
    save_every=SAVE_EVERY,
    report_every=REPORT_EVERY,
    fp16=FP16,
    # validation_function=validation_function
    # checkpoint="saved checkpoint path"
)

device is cuda


Iteration:   0%|          | 0/10688 [00:00<?, ?it/s]/home/ge75zam2/miniconda3/envs/finetune/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Iteration:   1%|          | 100/10688 [01:48<2:14:33,  1.31it/s]

timestamp: 21/06/2020 15:22:20, average loss: 8.350365, time duration: 108.646673,
                            number of examples in current reporting: 800, step 100
                            out of total 10000000


Iteration:   2%|▏         | 200/10688 [03:12<2:21:10,  1.24it/s]

timestamp: 21/06/2020 15:23:44, average loss: 6.040869, time duration: 83.839989,
                            number of examples in current reporting: 800, step 200
                            out of total 10000000


...

Iteration:  78%|███████▊  | 8296/10688 [1:58:12<33:50,  1.18it/s]

timestamp: 22/06/2020 13:29:55, average loss: 1.867685, time duration: 85.990846,
                            number of examples in current reporting: 800, step 93800
                            out of total 10000000


Iteration:  79%|███████▊  | 8396/10688 [1:59:39<32:11,  1.19it/s]

timestamp: 22/06/2020 13:31:22, average loss: 1.896232, time duration: 86.633912,
                            number of examples in current reporting: 800, step 93900
                            out of total 10000000


Iteration:  79%|███████▉  | 8419/10688 [1:59:58<31:01,  1.22it/s]

In [ ]:
summarizer.save_model(MAX_STEPS, os.path.join("/Users/gjke/Documents/uni/faktual", "bertsumabs.pt"))

## Model Evaluation

To run rouge evaluation, please refer to the section of compute_rouge_perl in [summarization_evaluation.ipynb](summarization_evaluation.ipynb) for setup.
For the settings in this notebook with QUICK_RUN=False, you should get ROUGE scores close to the following numbers: <br />
``
{'rouge-1': {'f': 0.34819639878321873,
             'p': 0.39977932634737307,
             'r': 0.34429079596863604},
 'rouge-2': {'f': 0.13919271352557894,
             'p': 0.16129965067780644,
             'r': 0.1372938054050938},
 'rouge-l': {'f': 0.2313282318854973,
             'p': 0.26664667422849747,
             'r': 0.22850294283399628}}
 ``
 
 Better performance can be achieved by increasing the MAX_STEPS.

In [ ]:

checkpoint = torch.load(os.path.join("/Users/gjke/Documents/uni/faktual", "bertsumabs.pt"), map_location="cpu")
summarizer = BertSumAbs(
    processor, cache_dir=CACHE_PATH, max_pos_length=MAX_POS, test=True
)
summarizer.model.load_checkpoint(checkpoint['model'])

In [ ]:
len(test_dataset)

In [ ]:
TEST_TOP_N = 10
if not QUICK_RUN:
    TEST_TOP_N = len(test_dataset)
TEST_TOP_N = 10
if NUM_GPUS:
    BATCH_SIZE = NUM_GPUS * BATCH_SIZE_PER_GPU
else:
    BATCH_SIZE = 1
    
shortened_dataset = test_dataset.shorten(top_n=TEST_TOP_N)
src = shortened_dataset.get_source()
reference_summaries = [" ".join(t).rstrip("\n") for t in shortened_dataset.get_target()]
generated_summaries = summarizer.predict(
    shortened_dataset, batch_size=BATCH_SIZE, num_gpus=NUM_GPUS
)
assert len(generated_summaries) == len(reference_summaries)

In [ ]:
shortened_dataset.get_source()[3]

In [ ]:
generated_summaries[3]

## Clean up temporary folders

In [ ]:
if os.path.exists(CACHE_PATH):
    shutil.rmtree(CACHE_PATH, ignore_errors=True)